In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sci
from utils import *

plt.rcParams["figure.figsize"] = (16,9)
sns.set(font_scale=1.6)
sns.set_style("whitegrid")

In [ ]:
df = context.catalog.load("fire_data_with_foehn_and_control_variables")

In [ ]:
foehn_prevalence_fire_start = (df["foehn_minutes_during_12_hours_after_start_of_fire"] >60).sum()/df["foehn_minutes_during_12_hours_after_start_of_fire"].count()
#foehn_prevalence_before_fire_start = (df_final["foehn_minutes_24_hour_before"] >60).sum()/df_final["foehn_minutes_24_hour_before"].count()

print(f"Foehn prevalent after start of fire (%): {foehn_prevalence_fire_start}")
#print(f"Foehn prevalent before start of fire (%): {foehn_prevalence_before_fire_start}")

In [ ]:
plot_binned_burned_area_after_fire_start(df, hours=6)
plot_binned_burned_area_after_fire_start(df, hours=12)

In [ ]:
plot_binned_burned_area_after_fire_start(df, hours=6, control_var="decade")
plot_binned_burned_area_after_fire_start(df, hours=12, control_var="decade")

In [ ]:
plot_binned_burned_area_after_fire_start(df, hours=6, control_var="fire_regime")
plot_binned_burned_area_after_fire_start(df, hours=12, control_var="fire_regime")

In [ ]:
plot_binned_burned_area_after_fire_start(df, hours=6, control_var="potential_foehn_species")
plot_binned_burned_area_after_fire_start(df, hours=12, control_var="potential_foehn_species")

# Statistical significance tests

In [ ]:
hours=12

In [ ]:
bins = [-0.001, 0.001] + [hours*10*i for i in range(1,6+1)]

no_foehn_mask = pd.cut(df[f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=bins) == pd.Interval(-0.001, 0.001)
no_foehn_fires = df.loc[no_foehn_mask, "total [ha]"]

for interval in sorted(pd.cut(df[f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=bins).value_counts().index[1:]):
    foehn_mask = pd.cut(df[f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=bins) == interval
    foehn_fires = df.loc[foehn_mask, "total [ha]"]

    print(interval, "\t",
          np.round(sci.stats.ranksums(no_foehn_fires, foehn_fires).pvalue,6), "\t",
          np.round(foehn_fires.median()/no_foehn_fires.median(), 2))

In [ ]:
# All fires
test_for_certain_aspect(df, hours)

In [ ]:
# For different fire regimes regimes
test_for_certain_aspect(df, 
                        hours=hours, 
                        control_var="fire_regime", 
                        categories = ["Winter anthropogenic", "Summer anthropogenic", "Summer natural"] 
                       )

In [ ]:
# For different foehn types
test_for_certain_aspect(df, 
                        hours=hours, 
                        control_var="potential_foehn_species", 
                        categories = ["North foehn", "South foehn"] )

In [ ]:
# Test for significant difference between North and South Foehn
category ="North foehn"
filter_category = df["potential_foehn_species"] == category
categories = pd.cut(df.loc[filter_category, f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=[-0.001, 0.001, 60*hours])
foehn_mask = categories== pd.Interval(0.001, 60*hours)

foehn_values_north = df.loc[foehn_mask & filter_category, "total [ha]"]

category ="South foehn"
filter_category = df["potential_foehn_species"] == category
categories = pd.cut(df.loc[filter_category, f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=[-0.001, 0.001, 60*hours])
foehn_mask = categories== pd.Interval(0.001, 60*hours)

foehn_values_south = df.loc[foehn_mask & filter_category, "total [ha]"]

print(category, "\t", np.round(sci.stats.ranksums(foehn_values_north, foehn_values_south).pvalue,6), "\t", np.round(foehn_values_north.median()/foehn_values_south.median(), 3))

In [ ]:
# For different decades
test_for_certain_aspect(df, 
                        hours=hours, 
                        control_var="decade", 
                        categories = ["[1980, 1989]", "[1990, 1999]","[2000, 2009]","[2010, 2019]"] )

In [ ]:
# Test for significant difference in foehn between decades
category ="[1990, 1999]"
filter_category = df["decade"] == category
categories = pd.cut(df.loc[filter_category, f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=[-0.001, 0.001, 60*hours])
foehn_mask = categories== pd.Interval(0.001, 60*hours)

foehn_values_1990 = df.loc[foehn_mask& filter_category, "total [ha]"]

category ="[2000, 2009]"
filter_category = df["decade"] == category
categories = pd.cut(df.loc[filter_category, f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=[-0.001, 0.001, 60*hours])
foehn_mask = categories== pd.Interval(0.001, 60*hours)

foehn_values_2000 = df.loc[foehn_mask& filter_category, "total [ha]"]

category ="[2010, 2019]"
filter_category = df["decade"] == category
categories = pd.cut(df.loc[filter_category, f'foehn_minutes_during_{hours}_hours_after_start_of_fire'], bins=[-0.001, 0.001, 60*hours])
foehn_mask = categories== pd.Interval(0.001, 60*hours)

foehn_values_2010 = df.loc[foehn_mask& filter_category, "total [ha]"]

print("1990", "\t", np.round(sci.stats.ranksums(foehn_values_1990, foehn_values_2000).pvalue,6), "\t", np.round(foehn_values_1990.median()/foehn_values_2000.median(), 3))
print("2000", "\t", np.round(sci.stats.ranksums(foehn_values_2000, foehn_values_2010).pvalue,6), "\t", np.round(foehn_values_2000.median()/foehn_values_2010.median(), 3))
print("2010", "\t", np.round(sci.stats.ranksums(foehn_values_1990, foehn_values_2010).pvalue,6), "\t", np.round(foehn_values_1990.median()/foehn_values_2010.median(), 3))

# Test for wind strength after start

In [ ]:
hours=6
foehn_mask = df[f'foehn_minutes_during_{hours}_hours_after_start_of_fire']>0
print(foehn_mask.sum())
df_fires_with_foehn = df.loc[foehn_mask, :]

strength_variable = f"FF_mean_during_{hours}_hours_after_start_of_fire"
df_fires_with_foehn[strength_variable].hist(alpha=0.5,bins=30)
if hours==6:
    if "FFX" in strength_variable:
        bins = [7.5,27,52.1,95]
    else:
        bins = [2.5,14.9,27.6,52]
else:
    if "FFX" in strength_variable:
        bins = [3,21,35,95]
    else:
        bins = [1,10.7,21,52]

In [ ]:
plt.figure(figsize=(16,9))
g = sns.boxplot(x=pd.cut(df_fires_with_foehn[strength_variable], bins=bins), y = df_fires_with_foehn["total [ha]"], color="tab:blue")
#sns.boxplot(x=pd.cut(df.loc[non_foehn_mask, f"FF_mean_during_{hours}_hours_after_start_of_fire"], bins=1), y = df.loc[non_foehn_mask, "total [ha]"])
plt.xlabel("Wind speed gusts FFX [km/h]")
plt.ylabel("Total burned area [ha]")
g.set_yscale("log")
plt.grid(True,which="both",ls="--",c='gray', alpha=0.5)
save_figure(f"BoxplotBurnedAreaOverFFXForFirst{hours}HoursAfterStart")


In [ ]:
weak_foehn_mask = pd.cut(df_fires_with_foehn[strength_variable], bins=bins) == pd.Interval(bins[0], bins[1])
weak_foehn_fires = df_fires_with_foehn.loc[weak_foehn_mask, "total [ha]"]

for interval in sorted(pd.cut(df_fires_with_foehn[strength_variable], bins=bins).value_counts().index[:]):
    stronger_foehn_mask = pd.cut(df_fires_with_foehn[strength_variable], bins=bins) == interval
    stronger_foehn_fires = df_fires_with_foehn.loc[stronger_foehn_mask,"total [ha]"]

    print(interval, "\t",
          np.round(sci.stats.ranksums(stronger_foehn_fires, weak_foehn_fires).pvalue,6), "\t",
          np.round(stronger_foehn_fires.median()/weak_foehn_fires.median(), 2))

weak_foehn_mask = pd.cut(df_fires_with_foehn[strength_variable], bins=bins) == pd.Interval(bins[1], bins[2])
weak_foehn_fires = df_fires_with_foehn.loc[weak_foehn_mask, "total [ha]"]

stronger_foehn_mask = pd.cut(df_fires_with_foehn[strength_variable], bins=bins) == pd.Interval(bins[2], bins[3])
stronger_foehn_fires = df_fires_with_foehn.loc[stronger_foehn_mask,"total [ha]"]

print(pd.Interval(bins[2], bins[3]), "\t",
      np.round(sci.stats.ranksums(stronger_foehn_fires, weak_foehn_fires).pvalue,6), "\t",
      np.round(stronger_foehn_fires.median()/weak_foehn_fires.median(), 2))

In [ ]:
# Before fire start
plot_binned_burned_area_before_fire_start(df, hours=24)
plot_binned_burned_area_before_fire_start(df, hours=48)

In [ ]:
hours=48
foehn_mask = (df_final[f'foehn_minutes_{hours}_hour_before']>60) & (df_final["potential_foehn_species"] == "North foehn")
df_fires_with_foehn = df_final.loc[foehn_mask, :]

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
strength_variable = f"TT_mean_{hours}_hour_before"
df_fires_with_foehn[strength_variable].hist(bins=30)

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
bins=[-60,-40,-20,0]
bins=[0,3,6,9,12,15]

df_binned = df_fires_with_foehn.groupby(pd.cut(df_fires_with_foehn[strength_variable], bins=bins)).count()
#sns.barplot(x=df_binned.index, y=df_binned["total [ha]"], color="tab:blue")

foehn_conditions_in_interval = np.zeros(len(bins)-1)
foehn_stations = [colname[0:3] for colname in df_foehn.filter(regex="foehn").columns.tolist()]
strength_abbr = strength_variable[0:2]

for station in north_foehn_stations:
    target_variable_where_foehn = df_foehn[f"{station}_{strength_abbr}"].where(df_foehn[f"{station}_foehn"]==1.0).rolling(6*hours, min_periods=1).mean()
    target_variable_where_no_foehn = df_foehn[f"{station}_{strength_abbr}"].where((df_foehn[f"{station}_foehn"]==0.0)).rolling(6*hours, min_periods=1).mean()
    strength_difference = target_variable_where_foehn - target_variable_where_no_foehn

    foehn_conditions_in_interval += np.array([((df_binned.index[i].left < strength_difference) &
                                             (strength_difference <df_binned.index[i].right)).sum() for i in range(len(df_binned.index))])
    print(foehn_conditions_in_interval)


plt.figure(figsize=(16,9))
sns.barplot(x=df_binned.index, y=(df_binned['total [ha]']/foehn_conditions_in_interval)/min((df_binned['total [ha]']/foehn_conditions_in_interval)), color="tab:blue")
plt.ylabel("Normalized count of fires")
plt.xlabel("Foehn temperature increase [K]")

fig_path = f'/home/chmony/Documents/Results/FoehnFireLink/NormalizedFireCountOverTemperatureForThe{hours}HoursBeforeStart.pdf'
plt.savefig(fig_path, bbox_inches='tight', dpi=200)
print(f"Saved figure at: {fig_path}")
# xticks, xlabels = plt.xticks()
# xlabels = [label.get_text() for label in xlabels]
# xlabels[0] = "[0.0" + xlabels[0][6:]
# plt.xticks(xticks, xlabels)
# save_figure(f"NormalizedFireCountOverFoehnMinutesForThe{hours}HoursBeforeStart")